## INCLUDES:

- LINEAR REGRESSION

- RANDOM FOREST REGRESSOR

- GRADIENT BOOSTING REGRESSOR

In [11]:
import pandas as pd
import numpy as np
import my_pickle as mp
import my_functions as mf
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from sklearn.ensemble import RandomForestRegressor

import my_resample as ms

import warnings
warnings.filterwarnings('ignore')


from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble.partial_dependence import plot_partial_dependence


from importlib import reload

import warnings
warnings.filterwarnings('ignore')

In [2]:
X = mp.unjson_it('data_X')
y = mp.unjson_it('data_y')['convo_length']
text_similarity_df = mp.unjson_it('data_text_similarity')
X['count_similarity'] = text_similarity_df['count_similarity']
X['tfidf_similarity'] = text_similarity_df['tfidf_similarity']

# PREPARE DATA

same train test split, resample, and scaling for everything

In [3]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X.as_matrix(), y.as_matrix(), random_state=17)

# resample
X_train, y_train = ms.oversample(X_train, y_train, .5)

# scale data
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# LINEAR REGRESSION

In [21]:
reload(mf)

<module 'my_functions' from '/Users/gandalf/Documents/Galvanize/MatchingService/my_functions.py'>

In [23]:
# fit model
model = LinearRegression()
model.fit(X_train, y_train)
print(model.score(X_train, y_train))

# make predictions
y_pred = model.predict(X_test)

# show metrics
mf.display_metrics(model, X_test, y_test)

# show importances
mf.display_importances_linear(model, X).head(5)

0.0625878380586

METRICS
Model recall: 0.18235294117647058
Model precision: 0.15048543689320387
Model accuracy: -435.63253416764707

CONFUSION MATRIX
[[2437  175]
 [ 139   31]]

key:
 TN   FP 
 FN   TP 

RECALL AND ACCURACY FOR DIFFERNET MODELS
recall     	 precision   	model
0.182352941176 	 0.150485436893 my model
0.0 	 	 0.0 		predict all zero
1.0 	 	 0.0611071171819 predict all one
0.529411764706 	 0.0664206642066 predict 50-50
0.0941176470588 0.0597014925373 predict 90-10


,feature,coefficient,abs_value,sign
61,sentence_count_receiver,"-4,982,871,002,043.86","4,982,871,002,043.86",-1.00
39,period_count_receiver,"4,353,332,221,797.84","4,353,332,221,797.84",1.00
7,exclaim_count_receiver,"1,104,183,292,974.34","1,104,183,292,974.34",1.00
43,question_count_receiver,"111,752,900,418.44","111,752,900,418.44",1.00
62,sentence_count_sender,"-22,761,863,653.90","22,761,863,653.90",-1.00


# RANDOM FOREST REGRESSOR

In [26]:
# fit model
model = RandomForestRegressor()
model.fit(X_train, y_train, sample_weight=None)

# make predictions
y_pred = model.predict(X_test)

# show metrics
mf.display_metrics(model, X_test, y_test)

# show importances
mf.display_importances_trees(model, X).head(5)


METRICS
Model recall: 0.3058823529411765
Model precision: 0.14285714285714285
Model accuracy: -454.1807229528874

CONFUSION MATRIX
[[2300  312]
 [ 118   52]]

key:
 TN   FP 
 FN   TP 

RECALL AND ACCURACY FOR DIFFERNET MODELS
recall     	 precision   	model
0.305882352941 	 0.142857142857 my model
0.0 	 	 0.0 		predict all zero
1.0 	 	 0.0611071171819 predict all one
0.488235294118 	 0.0583274771609 predict 50-50
0.117647058824 0.0716845878136 predict 90-10


,feature,coefficient
69,urgency_receiver,0.19
6,distance,0.12
72,tfidf_similarity,0.04
4,age_sender,0.04
21,hobbies_overlap,0.04


# GRADIENT BOOSTING REGRESSOR

In [29]:
# fit model
model = GradientBoostingRegressor()
model.fit(X_train, y_train, sample_weight=None)

# make predictions
y_pred = model.predict(X_test)

# show metrics
mf.display_metrics(model, X_test, y_test)

# show importances
mf.display_importances_trees(model, X).head(5)


METRICS
Model recall: 0.12352941176470589
Model precision: 0.17355371900826447
Model accuracy: -324.8349304483476

CONFUSION MATRIX
[[2512  100]
 [ 149   21]]

key:
 TN   FP 
 FN   TP 

RECALL AND ACCURACY FOR DIFFERNET MODELS
recall     	 precision   	model
0.123529411765 	 0.173553719008 my model
0.0 	 	 0.0 		predict all zero
1.0 	 	 0.0611071171819 predict all one
0.435294117647 	 0.0523338048091 predict 50-50
0.105882352941 0.0576923076923 predict 90-10


,feature,coefficient
6,distance,0.16
69,urgency_receiver,0.14
70,urgency_sender,0.10
72,tfidf_similarity,0.07
32,maxCost_receiver,0.07


### RESAMPLE AS A HYPERPARAMETER

In [ ]:
def all_together(X_df,resamp,cutoff):
    # GET DATA
    y = X_df.convo_length
    X = X_df.drop(['convo_length'], axis=1)

    # TRAIN TEST SPLIT
    X_train, X_test, y_train, y_test = train_test_split(X.as_matrix(), y.as_matrix(), random_state=17)

    # RESAMPLE
    X_train, y_train = ms.oversample(X_train, y_train, resamp)

    # SCALE DATA
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    # fit model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # make predictions
    y_pred = model.predict(X_test)
    
    y_pred_binary = [1 if x > cutoff else 0 for x in y_pred]
    y_test_binary = [1 if x > cutoff else 0 for x in y_test]
    
    return recall_score(y_test_binary, y_pred_binary), precision_score(y_test_binary, y_pred_binary)

In [ ]:
recall = []
precision = []
x_values = np.arange(.01,10,.01)
for x in x_values:
    r,p = all_together(X_df,x, 1)
    recall.append(r)
    precision.append(p)

In [ ]:
fig, ax = plt.subplots()
ax.plot(x_values, recall)
ax.plot(x_values, precision)
ax.set_title('RECALL AND PRECISION')
plt.show()

In [ ]:
def change_cutoff(cutoff, lotsa = True):
    y_pred_binary = [1 if x > cutoff else 0 for x in y_pred]
    y_test_binary = [1 if x > cutoff else 0 for x in y_test]

    if lotsa:
        print("\nMETRICS")
        print("Model recall: {}".format(recall_score(y_test_binary, y_pred_binary)))
        print("Model precision: {}".format(precision_score(y_test_binary, y_pred_binary)))
        print("Model accuracy: {}".format(model.score(X_test, y_test_binary)))

        print ("\nCONFUSION MATRIX")
        print (confusion_matrix(y_test_binary, y_pred_binary))
        print ("\nkey:")
        print (" TN   FP ")
        print (" FN   TP ")
    
    return recall_score(y_test_binary, y_pred_binary), precision_score(y_test_binary, y_pred_binary)

change_cutoff(1.5)

In [ ]:
def plot_recall_precision():
    recall = []
    precision = []
    x_values = np.arange(0,10,.1)
    for cutoff in x_values:
        r,p = change_cutoff(cutoff,False)
        recall.append(r)
        precision.append(p)
    fig, ax = plt.subplots()
    ax.plot(x_values, recall)
    ax.plot(x_values, precision)
    ax.set_title('RECALL AND PRECISION')
    plt.show()
plot_recall_precision()

In [ ]:
plot_partial_dependence(model, X_train, feature_importances[-6:], X.columns, n_jobs=-1,figsize = (16,8))
plt.show()

In [ ]:
fig, axs = plot_partial_dependence(model, X_train, [71]) 
plt.show()

In [ ]:
def change_cutoff(cutoff, lotsa = True):
    y_pred_binary = [1 if x > cutoff else 0 for x in y_pred]
    y_test_binary = [1 if x > cutoff else 0 for x in y_test]

    if lotsa:
        print("\nMETRICS")
        print("Model recall: {}".format(recall_score(y_test_binary, y_pred_binary)))
        print("Model precision: {}".format(precision_score(y_test_binary, y_pred_binary)))
        print("Model accuracy: {}".format(model.score(X_test, y_test_binary)))

        print ("\nCONFUSION MATRIX")
        print (confusion_matrix(y_test_binary, y_pred_binary))
        print ("\nkey:")
        print (" TN   FP ")
        print (" FN   TP ")
    
    return recall_score(y_test_binary, y_pred_binary), precision_score(y_test_binary, y_pred_binary)

change_cutoff(1)

In [ ]:
def plot_recall_precision():
    recall = []
    precision = []
    x_values = np.arange(0,10,.1)
    for cutoff in x_values:
        r,p = change_cutoff(cutoff,False)
        recall.append(r)
        precision.append(p)
    fig, ax = plt.subplots()
    ax.plot(x_values, recall)
    ax.plot(x_values, precision)
    ax.set_title('RECALL AND PRECISION')
    plt.show()
plot_recall_precision()